In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#   print(dirname)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install torchsummary

In [3]:
traindir = '../input/chest-xray-pneumonia/chest_xray/train'
testdir = '../input/chest-xray-pneumonia/chest_xray/test'
valdir = '../input/chest-xray-pneumonia/chest_xray/val'

In [4]:
files = next(os.walk(traindir+'/PNEUMONIA'))[2]
train_count_p = len(files)
files = next(os.walk(traindir+'/NORMAL'))[2]
train_count_n = len(files)
print("P test Number = ", train_count_p)
print("N test number = ", train_count_n)

files = next(os.walk(testdir+'/PNEUMONIA'))[2]
test_count_p = len(files)
files = next(os.walk(testdir+'/NORMAL'))[2]
test_count_n = len(files)
print("P test Number = ", test_count_p)
print("N test number = ", test_count_n)

files = next(os.walk(valdir+'/PNEUMONIA'))[2]
val_count_p = len(files)
files = next(os.walk(valdir+'/NORMAL'))[2]
val_count_n = len(files)
print("P test Number = ", val_count_p)
print("N test number = ", val_count_n)

In [5]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 5))

for i in range(9):
    plt.subplot(3,3,i+1)
    img = plt.imread(os.path.join(testdir+'/PNEUMONIA', os.listdir(testdir+'/PNEUMONIA')[i]))
    plt.title('Pneumonia')
    plt.imshow(img, cmap='gray')
    plt.axis('off')

In [6]:
plt.figure(figsize=(15, 5))

for i in range(9):
    plt.subplot(3,3,i+1)
    img = plt.imread(os.path.join(testdir+'/NORMAL', os.listdir(testdir+'/NORMAL')[i]))
    plt.title('Normal')
    plt.imshow(img, cmap = 'gray')
    plt.axis('off')

In [7]:
import glob

train_p = glob.glob(traindir+'/PNEUMONIA/*.jpeg')
train_n = glob.glob(traindir+'/NORMAL/*.jpeg')

In [8]:
import pandas as pd
import numpy as np

In [9]:
 data = pd.DataFrame(np.concatenate([[0]*len(train_n) , [1]*len(train_p)]),columns=["class"])

In [10]:
import seaborn as sns
plt.figure(figsize=(15,10))
sns.countplot(data['class'], data=data, palette ='rocket')
plt.title('PNEUMONIA VS NORMAL')
plt.show()

In [11]:
import torch
from torchvision import transforms
train_trnsf = transforms.Compose([transforms.RandomRotation(degrees=(-20,+20)),
                                 transforms.Resize((256,256)),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

test_trnsf = transforms.Compose([transforms.Resize((256,256)),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

In [12]:
from torchvision.datasets import ImageFolder
train_ds = ImageFolder(traindir, train_trnsf)
test_ds = ImageFolder(testdir, test_trnsf)
val_ds = ImageFolder(valdir, test_trnsf)

In [13]:
img, labels = train_ds[0]
img.shape
train_ds.classes

In [14]:
class_names = train_ds.classes
print(class_names)
print(train_ds.class_to_idx)

In [15]:
from torch.utils.data import DataLoader
batch_size = 64
trainloader = DataLoader(train_ds, batch_size, shuffle = True, num_workers=2, pin_memory=True)
testloader = DataLoader(test_ds, batch_size, shuffle = True, num_workers=2, pin_memory=True)
valloader = DataLoader(val_ds, batch_size*2, shuffle = True, num_workers=2, pin_memory=True)

In [16]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x,device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
    
    def __len__(self):
        return len(self.dl)


In [17]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
class SimpleResidualBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 3, kernel_size = 3, stride = 1, padding = 1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels = 3, out_channels = 3, kernel_size = 3,stride = 1, padding = 1)
        self.relu2 = nn.ReLU()
        
    def forward(self,x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.conv2(out)
        out = self.relu2(out)
        return out + x

In [18]:
device = get_default_device()
device

In [19]:
train_dl = DeviceDataLoader(trainloader, device)
valid_dl = DeviceDataLoader(valloader, device)

In [20]:
simple_resnet = to_device(SimpleResidualBlock(), device)
count = 0

for images,labels in train_dl:
    out = simple_resnet(images)
    print(out.shape)
    print(count+1)
    break
    
del simple_resnet, images, labels
torch.cuda.empty_cache()


In [21]:
def accuracy(outputs,labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item()/len(preds))

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return {'val_loss': loss.detach(), 'val_acc': acc}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
        epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc']))

# Trying ResNet 9 model 

But cannot compromise with the size of the x ray image since there are very minute details in the X-ray that have to be taken into consideration. So ended up requiring large number of neurons in the final neural network.
This resulted in high fluctuation of accuracy during training.




In [ ]:
def conv_block(in_channels,  out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
             nn.BatchNorm2d(out_channels),
             nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)
    
class ResNet9(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        
        self.conv1 = conv_block(in_channels, 64, pool=True) # 64 x 64 x 128 x 128
        self.conv2 = conv_block(64, 128, pool=True) # 64 x 128 x 64 x 64
        self.res1 = nn.Sequential(conv_block(128,128), conv_block(128,128)) # 64 x 128 x 64 x 64
        
        self.conv3 = conv_block(128,256, pool=True) # 64 x 256 x 32 x 32
        self.conv4 = conv_block(256,512,pool=True) # 64 x 512 x 16 x 16
        self.res2 = nn.Sequential(conv_block(512,512), conv_block(512,512)) # 64 x 512 x 16 x 16
        
        self.classifier = nn.Sequential(nn.MaxPool2d(4),
                                        nn.Flatten(),
                                        nn.Dropout(0.2),
                                        nn.Linear(8192, num_classes)) # 512 x 4 x 4
        
    def forward(self, xb):
            out = self.conv1(xb)
            out = self.conv2(out)
            out = self.res1(out) + out
            out = self.conv3(out)
            out = self.conv4(out)
            out = self.res2(out) + out
            out = self.classifier(out)
            return out

> **Tried decreasing the number of channels and removing a Max pooling layer in the beginning. 
> Received the same number of parameters in the end (8192), so increased the number of layers in the neural network.
> There was longer training time and the accuracy kept fluctuating again**


In [ ]:
def conv_block(in_channels,  out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
             nn.BatchNorm2d(out_channels),
             nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)
    
class ResNet9(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        
        self.conv1 = conv_block(in_channels, 16) # 64 x 16 x 256 x 256
        self.conv2 = conv_block(16, 32, pool=True) # 64 x 32 x 64 x 64
        self.res1 = nn.Sequential(conv_block(32,32), conv_block(32,32)) # 64 x 32 x 64 x 64
        
        self.conv3 = conv_block(32,64, pool=True) # 64 x 64 x 32 x 32
        self.conv4 = conv_block(64,128,pool=True) # 64 x 128 x 16 x 16
        self.res2 = nn.Sequential(conv_block(128,128), conv_block(128,128)) # 64 x 128 x 16 x 16
        
        self.classifier = nn.Sequential(nn.MaxPool2d(4),
                                        nn.Flatten(),
                                        nn.Dropout(0.2),
                                        nn.Linear(8192, 4096),
                                       nn.Linear(4096,128),
                                       nn.Linear(128, num_classes)) # 128 x 4 x 4
        
    def forward(self, xb):
            out = self.conv1(xb)
            out = self.conv2(out)
            out = self.res1(out) + out
            out = self.conv3(out)
            out = self.conv4(out)
            out = self.res2(out) + out
            out = self.classifier(out)
            return out

In [ ]:
model = to_device(ResNet9(3,2), device)
model

Now using only CNN layers and avoiding using the ResNet9 architecture.


In [22]:
def accuracy(outputs,labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item()/len(preds))

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return {'val_loss': loss.detach(), 'val_acc': acc}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
        epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc']))

In [ ]:
def conv_block(in_channels,  out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
             nn.BatchNorm2d(out_channels),
             nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class CNN(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.conv1 = conv_block(in_channels, 32) # 64 x 32 x 256 x 256
        self.conv2 = conv_block(32, 64, pool=True) # 64 x 64 x 64 x 64
                
        self.conv3 = conv_block(64,128, pool=True) # 64 x 128 x 32 x 32
        self.conv4 = conv_block(128,256,pool=True) # 64 x 256 x 16 x 16
        
        
        self.classifier = nn.Sequential(nn.MaxPool2d(4),
                                        nn.Flatten(),
                                        nn.Dropout(0.2),
                                        nn.Linear(16384, 4096),
                                       nn.Linear(4096,128),
                                       nn.Linear(128, num_classes)) # 128 x 4 x 4
        
    def forward(self, xb):
            out = self.conv1(xb)
            out = self.conv2(out)
            
            out = self.conv3(out)
            out = self.conv4(out)
            
            out = self.classifier(out)
            return out

In [ ]:
model = to_device(CNN(3,2), device)
model

In [ ]:
from torchsummary import summary
summary(model, input_size=(3, 256, 256))

In [ ]:
@torch.no_grad()

def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
    
def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, steps_per_epoch=len(train_loader))
    
    for epoch in range(epochs):
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
                
            optimizer.step()
            optimizer.zero_grad()
            
            lrs.append(get_lr(optimizer))
            sched.step()
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
history = [evaluate(model, valid_dl)]
history

In [ ]:
[model.validation_step(batch) for batch in valid_dl]


In [ ]:
epochs = 8
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [ ]:
%%time
history += fit_one_cycle(epochs, max_lr, model, train_dl, valid_dl,
                        grad_clip = grad_clip,
                        weight_decay = weight_decay,
                        opt_func = opt_func)

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs Number of epochs')

In [ ]:
plot_accuracies(history)

In [ ]:
def plot_losses(history):
    train_losses = [x.get('train loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs No. of epochs')

In [ ]:
plot_losses(history)

In [ ]:
def plot_lrs(history):
    lrs = np.concatenate([x.get('lrs',[]) for x in history])
    plt.plot(lrs)
    plt.xlabel('Batch No.')
    plt.ylabel('Learning Rate')
    plt.title('Learning Rate vs Batch no.');

In [ ]:
plot_lrs(history)

In [ ]:
def predict_image(img, model):
    xb = to_device(img.unsqueeze(0), device)
    yb = model(xb)
    _, preds = torch.max(yb, dim=1)
    return train_ds.classes[preds[0].item()]

In [ ]:
img, label = val_ds[0]
plt.imshow(img.permute(1,2,0).clamp(0,1))
print('Label', train_ds.classes[label], ',Predicted', predict_image(img, model))

In [ ]:
img, label = val_ds[11]
plt.imshow(img.permute(1,2,0))
print('Label:', val_ds.classes[label])

In [ ]:
img, label = val_ds[6]
plt.imshow(img.permute(1, 2, 0))
print('Label:', train_ds.classes[label], ', Predicted:', predict_image(img, model))

In [ ]:
torch.save(model.state_dict(), 'resnet9.pth')